# RFM ile MÜŞTERİ SEGMENTASYONU

**Veri Seti Hikayesi**

https://archive.ics.uci.edu/ml/datasets/Online+Retail+II

Online Retail II isimli veri seti İngiltere merkezli online bir satış mağazasının 01/12/2009 - 09/12/2011 tarihleri arasındaki satışlarını içeriyor.

Bu şirket hediyelik eşya satıyor.

**Değişkenler**

- InvoiceNo: Fatura numarası. Her işleme yani faturaya ait eşsiz numara. Eğer bu kod C ile başlıyorsa işlemin iptal edildiğini ifade eder.
- StockCode: Ürün kodu. Her bir ürün için eşsiz numara.
- Description: Ürün ismi
- Quantity: Ürün adedi. Faturalardaki ürünlerden kaçar tane satıldığını ifade etmektedir.
- InvoiceDate: Fatura tarihi ve zamanı. 
- UnitPrice: Ürün fiyatı (Sterlin cinsinden)
- CustomerID: Eşsiz müşteri numarası
- Country: Ülke ismi. Müşterinin yaşadığı ülke.

## 1. Data Understanding

In [4]:
# Kütüphane kurulumları...
import pandas as pd
import numpy as np
import seaborn as sns

# Virgülden sonra gösterilecek olan sayı. (Opsiyonel)
pd.set_option('display.float_format', lambda x: '%.0f' % x)

In [5]:
# Veri setindeki 2010-2011 tarihleri aralığını seçiyorum.
df_2010_2011 = pd.read_excel("online_retail_II.xlsx", sheet_name = "Year 2010-2011")

In [6]:
df = df_2010_2011.copy()

In [7]:
#İlk 10 veriyi getirelim.
df.head(10)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,3,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,3,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3,17850,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,8,17850,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4,17850,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,2,17850,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,2,17850,United Kingdom
9,536368,22960,JAM MAKING SET WITH JARS,6,2010-12-01 08:34:00,4,13047,United Kingdom


In [8]:
# Gözlem ve sütun sayısı
df.shape

(541910, 8)

In [9]:
# Dataframe'e ait kısa özet alalım.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 8 columns):
Invoice        541910 non-null object
StockCode      541910 non-null object
Description    540456 non-null object
Quantity       541910 non-null int64
InvoiceDate    541910 non-null datetime64[ns]
Price          541910 non-null float64
Customer ID    406830 non-null float64
Country        541910 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


### DataFrame anlamaya yönelik sorular

In [10]:
#essiz urun sayisi nedir?
df["Description"].nunique()

4223

In [11]:
#hangi urunden kacar tane var?
# value_counts() : Kategorik değişkenlerin sınıflarını saydırır.
df["Description"].value_counts().head()

WHITE HANGING HEART T-LIGHT HOLDER    2369
REGENCY CAKESTAND 3 TIER              2200
JUMBO BAG RED RETROSPOT               2159
PARTY BUNTING                         1727
LUNCH BAG RED RETROSPOT               1638
Name: Description, dtype: int64

In [12]:
#en cok siparis edilen urunler hangisi?
df.groupby("Description").agg({"Quantity" : "sum"}).sort_values("Quantity", ascending = False).head()

,Quantity
Description,
WORLD WAR 2 GLIDERS ASSTD DESIGNS,53847
JUMBO BAG RED RETROSPOT,47363
ASSORTED COLOUR BIRD ORNAMENT,36381
POPCORN HOLDER,36334
PACK OF 72 RETROSPOT CAKE CASES,36039


In [13]:
#toplam kac fatura kesilmiştir?
df["Invoice"].nunique()

25900

In [14]:
#fatura basina toplam kac para kazanilmistir? (iki değişkeni çarparak yeni bir değişken oluşturmak gerekmektedir.)
# Burada Feature Engineering yaptık. Değişken türettik.
''' Feature Engineering : Veri setinde görüken ya da gözükmeyen yeni değişkenler türetmektir. 
Bazen bu mevcut değişkenler üzerinden bazende yapısal olmayan farklı kaynaklardan olur.'''

df["TotalPrice"] = df["Quantity"]*df["Price"]

In [15]:
df.groupby("Invoice").agg({"TotalPrice":"sum"}).head()

,TotalPrice
Invoice,
536365,139
536366,22
536367,279
536368,70
536369,18


In [16]:
#en pahalı ürünler hangileri?
df.sort_values("Price", ascending = False).head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
222681,C556445,M,Manual,-1,2011-06-10 15:31:00,38970,15098,United Kingdom,-38970
524602,C580605,AMAZONFEE,AMAZON FEE,-1,2011-12-05 11:36:00,17836,nan,United Kingdom,-17836
43702,C540117,AMAZONFEE,AMAZON FEE,-1,2011-01-05 09:55:00,16888,nan,United Kingdom,-16888
43703,C540118,AMAZONFEE,AMAZON FEE,-1,2011-01-05 09:57:00,16454,nan,United Kingdom,-16454
16356,C537651,AMAZONFEE,AMAZON FEE,-1,2010-12-07 15:49:00,13541,nan,United Kingdom,-13541


''' Dikkat edilmesi gerekenler :
    "Invoce" değişenine odaklanın ve C ile başlayanların iade ürünler olduğunu bilin ve onlardan kurtulmaya çalışın'''

In [17]:
#hangi ulkeden kac siparis geldi?
df["Country"].value_counts()

United Kingdom          495478
Germany                   9495
France                    8558
EIRE                      8196
Spain                     2533
Netherlands               2371
Belgium                   2069
Switzerland               2002
Portugal                  1519
Australia                 1259
Norway                    1086
Italy                      803
Channel Islands            758
Finland                    695
Cyprus                     622
Sweden                     462
Unspecified                446
Austria                    401
Denmark                    389
Japan                      358
Poland                     341
Israel                     297
USA                        291
Hong Kong                  288
Singapore                  229
Iceland                    182
Canada                     151
Greece                     146
Malta                      127
United Arab Emirates        68
European Community          61
RSA                         58
Lebanon 

In [18]:
#hangi ulke ne kadar kazandırdı?
df.groupby("Country").agg({"TotalPrice":"sum"}).sort_values("TotalPrice", ascending = False).head()

,TotalPrice
Country,
United Kingdom,8187806
Netherlands,284662
EIRE,263277
Germany,221698
France,197422


In [19]:
#en cok iade alan urun hangisidir?
df[df["Invoice"].astype(str).str.startswith("C")].sort_values("Quantity").head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
540422,C581484,23843,"PAPER CRAFT , LITTLE BIRDIE",-80995,2011-12-09 09:27:00,2,16446,United Kingdom,-168470
61624,C541433,23166,MEDIUM CERAMIC TOP STORAGE JAR,-74215,2011-01-18 10:17:00,1,12346,United Kingdom,-77184
4268,C536757,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,-9360,2010-12-02 14:23:00,0,15838,United Kingdom,-281
160145,C550456,21108,FAIRY CAKE FLANNEL ASSORTED COLOUR,-3114,2011-04-18 13:08:00,2,15749,United Kingdom,-6539
160144,C550456,21175,GIN + TONIC DIET METAL SIGN,-2000,2011-04-18 13:08:00,2,15749,United Kingdom,-3700


In [20]:
# İade ürünleri silmemiz gerektiğini söylemiştik, peki nasıl?
# Bu kod ile C olmayanları al ve dataframe tekrar atama yap dedik.
df=df[~df["Invoice"].astype(str).str.startswith("C")] 
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,3,17850,United Kingdom,15
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3,17850,United Kingdom,20
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,3,17850,United Kingdom,22
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3,17850,United Kingdom,20
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3,17850,United Kingdom,20


# 2. Data Preparation

### Eksik Değerler

In [21]:
# Kaç eksik değer var?
df.isnull().sum()

Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    134697
Country             0
TotalPrice          0
dtype: int64

In [22]:
# Modelleme yapmadığımız için eksik değerlerin bir önemi yok, onlardan kurtuluyorum.
# inplace = True : Kalıcı silme yapar.
df.dropna(inplace = True)

In [23]:
#Hiç eksik değer kaldı mı?
df.isnull().values.any()

False

In [24]:
# Ne kadar gözlem birimi gitti diye bakıyoruz.
df.shape

(397925, 9)

### Aykırı Değerler

ML işlemi yapmayacağımız için aykırı değerlerinde anlamı düşmüş oldu. Aykırı değerlere işlem yapmıyoruz çünkü RFM Skorlarını aykırı değerlere göre oluşturacağız.

In [25]:
# Basit bir bakış açısıyla aykırı değerlere değinelim.
df.describe([0.01,0.05,0.10,0.25,0.50,0.75,0.90,0.95, 0.99]).T

,count,mean,std,min,1%,5%,10%,25%,50%,75%,90%,95%,99%,max
Quantity,397925,13,180,1,1,1,1,2,6,12,24,36,120,80995
Price,397925,3,22,0,0,0,1,1,2,4,6,8,15,8143
Customer ID,397925,15294,1713,12346,12415,12627,12883,13969,15159,16795,17725,17912,18211,18287
TotalPrice,397925,22,309,0,1,1,2,5,12,20,35,68,202,168470


In [26]:
# Aykırı değerleri bulmaya yarayan kod parçaçığı.
for feature in ["Quantity","Price","TotalPrice"]:

    Q1 = df[feature].quantile(0.01)
    Q3 = df[feature].quantile(0.99)
    IQR = Q3-Q1
    upper = Q3 + 1.5*IQR
    lower = Q1 - 1.5*IQR

    if df[(df[feature] > upper) | (df[feature] < lower)].any(axis=None):
        print(feature,"yes")
        print(df[(df[feature] > upper) | (df[feature] < lower)].shape[0])
    else:
        print(feature, "no")

Quantity yes
963
Price yes
661
TotalPrice yes
903


# 3. RFM Skorları ile Müşteri Segmentasyonu

-  **Recency (yenilik):** Müşterinin son satın almasından ya da temasından bugüne kadar geçen ki süre
    
    -- (Bugünün tarihi - Son satın alma)

-  **Frequency (Sıklık):** Toplam satın alma sayısı.

-  **Monetary (Parasal Değer):** Müşterinin yaptığı toplam harcama. 

In [61]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,3,17850,United Kingdom,15
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3,17850,United Kingdom,20
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,3,17850,United Kingdom,22
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3,17850,United Kingdom,20
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3,17850,United Kingdom,20


In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 397925 entries, 0 to 541909
Data columns (total 9 columns):
Invoice        397925 non-null object
StockCode      397925 non-null object
Description    397925 non-null object
Quantity       397925 non-null int64
InvoiceDate    397925 non-null datetime64[ns]
Price          397925 non-null float64
Customer ID    397925 non-null float64
Country        397925 non-null object
TotalPrice     397925 non-null float64
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 30.4+ MB


## Recency Hesaplama

In [31]:
# Veri setindeki minimum değer.
df["InvoiceDate"].min()

Timestamp('2010-12-01 08:26:00')

In [32]:
# Veri setine max değeri.
df["InvoiceDate"].max()

Timestamp('2011-12-09 12:50:00')

In [33]:
# Veri setine göre bu max değeri bugünün tarihi olarak atayalım.
import datetime as dt   
today_date = dt.datetime(2011,12,9)
today_date 

datetime.datetime(2011, 12, 9, 0, 0)

Müşterinin son satın almasından bugüne kadar geçen süreyi nasıl tanımlarım? Bugünün tarihini biliyoruz. Son satın alma tarihi kalıyor geriye. 
Bunu da müşteri özelinde yapabiliriz.

In [34]:
df.groupby("Customer ID").agg({"InvoiceDate":"max"}).head()

,InvoiceDate
Customer ID,
12346,2011-01-18 10:01:00
12347,2011-12-07 15:52:00
12348,2011-09-25 13:13:00
12349,2011-11-21 09:51:00
12350,2011-02-02 16:01:00


In [35]:
# Customer ID integer çevirdik.
df["Customer ID"] = df["Customer ID"].astype(int)

In [36]:
# Bugünden müşteri özelinde son satın alınma tarihini çıkarma işlemi yaptık.
(today_date - df.groupby("Customer ID").agg({"InvoiceDate":"max"})).head()

,InvoiceDate
Customer ID,
12346,324 days 13:59:00
12347,1 days 08:08:00
12348,74 days 10:47:00
12349,17 days 14:09:00
12350,309 days 07:59:00


Bu şekilde Recency değerlerimizi bulmuş olduk. InvoiceDate yerine Recency yazabiliriz artık. Dipnot : InvoiceDate sütunundaki saatleri kaldırırsak daha güzel bir dataframe elde ederiz. Bahsettiğim işlemler aşağıdaki kodda.

In [50]:
temp_df = (today_date - df.groupby("Customer ID").agg({"InvoiceDate":"max"}))

In [51]:
temp_df.rename(columns={"InvoiceDate": "Recency"}, inplace = True)

In [55]:
temp_df.head()

,Recency
Customer ID,
12346,324 days 13:59:00
12347,1 days 08:08:00
12348,74 days 10:47:00
12349,17 days 14:09:00
12350,309 days 07:59:00


In [58]:
# Saatlerden kurtulma işlemi. Sadece günleri al dedik.
recency_df = temp_df["Recency"].apply(lambda x: x.days)

In [59]:
#Artık herbir müşterinin analiz yaptığımdan günden en son satın alma gününe kadar geçen gün olarak süresi elimde.
recency_df.head()

Customer ID
12346    324
12347      1
12348     74
12349     17
12350    309
Name: Recency, dtype: int64

## Frequency Hesaplama

In [63]:
# Herbir müşterinin kaç faturası var. Bu müşterilerin toplam işlem sayısı önemli. Bu dataframe üzerinden bir işlem daha gerçekleştirmek gerekir.
temp_df = df.groupby(["Customer ID","Invoice"]).agg({"Invoice":"count"})

In [64]:
temp_df.head()

Invoice
Customer ID Invoice         
12346       541431         1
12347       537626        31
            542237        29
            549222        24
            556201        18

In [65]:
# Herbir müşteri özelinde toplam yaptığı işlemler.
temp_df.groupby("Customer ID").agg({"Invoice":"sum"}).head()

,Invoice
Customer ID,
12346,1
12347,182
12348,31
12349,73
12350,17


In [66]:
freq_df = temp_df.groupby("Customer ID").agg({"Invoice":"sum"})
freq_df.rename(columns={"Invoice": "Frequency"}, inplace = True)
freq_df.head()

,Frequency
Customer ID,
12346,1
12347,182
12348,31
12349,73
12350,17


## Monetary Hesaplanması

In [67]:
monetary_df = df.groupby("Customer ID").agg({"TotalPrice":"sum"})
monetary_df.head()

,TotalPrice
Customer ID,
12346,77184
12347,4310
12348,1797
12349,1758
12350,334


In [68]:
monetary_df.rename(columns={"TotalPrice": "Monetary"}, inplace = True)

In [71]:
print(recency_df.shape,freq_df.shape,monetary_df.shape)

(4339,) (4339, 1) (4339, 1)


In [72]:
rfm = pd.concat([recency_df, freq_df, monetary_df],  axis=1)
rfm.head()

,Recency,Frequency,Monetary
Customer ID,,,
12346,324,1,77184
12347,1,182,4310
12348,74,31,1797
12349,17,73,1758
12350,309,17,334


### RFM Değerlerini Skorlama

Receny'de küçük değerler, Frequency ve Monetary'de büyük değerler bizim için iyidir.

In [74]:
# qcut : Küçükten büyüğe sıralayıp, quartilerlara bölme işlemi yapar. Burada bizim için önemli olan küçük değerlerdir bu yüzden onlara 5, büyük olan değerlere 1 atıyoruz
rfm["RecencyScore"] = pd.qcut(rfm["Recency"],5,labels = [5,4,3,2,1])

In [75]:
rfm["FrequencyScore"]= pd.qcut(rfm["Frequency"].rank(method="first"),5, labels=[1,2,3,4,5])


In [76]:
rfm["MonetaryScore"] = pd.qcut(rfm['Monetary'], 5, labels = [1, 2, 3, 4, 5])


In [77]:
rfm.head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore
Customer ID,,,,,,
12346,324,1,77184,1,1,5
12347,1,182,4310,5,5,5
12348,74,31,1797,2,3,4
12349,17,73,1758,4,4,4
12350,309,17,334,1,2,2


In [78]:
# Skorları oluşturmak için astype ile değişkenleri stringe çevirmeliyiz ki bu değerleri yan yana koyduğumda anlamı olabilsin.
(rfm['RecencyScore'].astype(str) + 
 rfm['FrequencyScore'].astype(str) + 
 rfm['MonetaryScore'].astype(str)).head()

Customer ID
12346    115
12347    555
12348    234
12349    444
12350    122
dtype: object

In [79]:
# Yukarı yapılan işlemi kalıcı olarak dataframe ekledim
rfm["RFM_SCORE"] = rfm['RecencyScore'].astype(str) + rfm['FrequencyScore'].astype(str) + rfm['MonetaryScore'].astype(str)

In [80]:
rfm.head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE
Customer ID,,,,,,,
12346,324,1,77184,1,1,5,115
12347,1,182,4310,5,5,5,555
12348,74,31,1797,2,3,4,234
12349,17,73,1758,4,4,4,444
12350,309,17,334,1,2,2,122


In [81]:
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
Recency,4339,91,100,-1,16,49,140,372
Frequency,4339,92,229,1,17,41,100,7847
Monetary,4339,2054,8988,0,307,674,1662,280206


İşlem özelinde seçimler yapabiliriz.

In [82]:
# Manuel olarak "Champions" sınıfını getir dersek:
rfm[rfm["RFM_SCORE"] == "555"].head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE
Customer ID,,,,,,,
12347,1,182,4310,5,5,5,555
12362,2,266,5226,5,5,5,555
12417,2,192,3649,5,5,5,555
12433,-1,420,13376,5,5,5,555
12437,0,200,4951,5,5,5,555


In [83]:
# En kötü müşterilerimiz: 
rfm[rfm["RFM_SCORE"] == "111"].head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE
Customer ID,,,,,,,
12353,203,4,89,1,1,1,111
12361,286,10,190,1,1,1,111
12401,302,5,84,1,1,1,111
12402,322,11,226,1,1,1,111
12441,365,11,174,1,1,1,111


Bunu otomatik hale getirmek için tüm segmentlere uygun sınıfları tek seferde belirtelim.

### Müşteri Segmentleri

In [100]:
seg_map = {
    r'[1-2][1-2]': 'Hibernating',
    r'[1-2][3-4]': 'At Risk',
    r'[1-2]5': 'Can\'t Loose',
    r'3[1-2]': 'About to Sleep',
    r'33': 'Need Attention',
    r'[3-4][4-5]': 'Loyal Customers',
    r'41': 'Promising',
    r'51': 'New Customers',
    r'[4-5][2-3]': 'Potential Loyalists',
    r'5[4-5]': 'Champions'
}

In [102]:
# Segment değişkenine Recency ve Frequency verdik. Bu iki metrik özelinde tablomuzu oluşturuyoruz. Monetary, Frequency ile aynı sayılabilir yakın değerler olduğu için.
rfm['Segment'] = rfm['RecencyScore'].astype(str) + rfm['FrequencyScore'].astype(str)
rfm['Segment'] = rfm['Segment'].replace(seg_map, regex=True)
rfm.head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE,Segment
Customer ID,,,,,,,,
12346,324,1,77184,1,1,5,115,Hibernating
12347,1,182,4310,5,5,5,555,Champions
12348,74,31,1797,2,3,4,234,At Risk
12349,17,73,1758,4,4,4,444,Loyal Customers
12350,309,17,334,1,2,2,122,Hibernating


In [104]:
# Grupları raporlayalım.
# Yukarıda müşterilere odaklanmıştık şimdi ise segmentlere.
rfm[["Segment","Recency","Frequency","Monetary"]].groupby("Segment").agg(["mean","count"])

Recency       Frequency       Monetary      
                       mean count      mean count     mean count
Segment                                                         
About to Sleep           51   316        16   316      449   316
At Risk                 165   583        56   583      986   583
Can't Loose             142    84       182    84     2371    84
Champions                 4   607       289   607     6950   607
Hibernating             208  1060        13  1060      537  1060
Loyal Customers          32   821       157   821     2835   821
Need Attention           51   210        41   210      846   210
New Customers             5    55         7    55     3742    55
Potential Loyalists      15   498        34   498      905   498
Promising                21   105         7   105      438   105

## Reporting

-  **Champions**

Champions sınıfı incelendiğinde:

-- Receny ortalaması 4, bu son zamanlarda alışveriş yapmış ve bizimle teması olmuş,

-- Frequency ortalaması yaklaşık 289, ciddi miktarda ürün satın almış,

-- Monetary ortalaması 6950.14, herbir ürün için yüksek miktarda para bırakmış yorumları yaparız.

    -- Son zamanlarda, sık alışveriş yapmış ve çok para harcamış olan bu sınıfın çok üstüne düşmemiz gerekmez. Özel günlerde mesaj atılabilir, yeni ürünler için bilgilendirilebilir, ödüllendirmek için promosyon gönderilebiliriz. 

- **Need Attention**

Need Attention sınıfı incelendiğinde:

-- Receny ortalaması 51, çok ortalarda bir teması var,

--  Frequency ortalaması 41, bir öncekine ciddi bir ürün satın alma azalması var ama yinede diğerlerine göre ortalarda,

--  Monetary ortalaması 846, 41 ürün için verdiği toplam miktar şeklinde yorumlar yaparız.

    -- En dikkat edilmesi gereken sınıf. İlgilenilmezse daha alt sınıfa kayabilir ya da gerekli müdahaleler yapılırsa Loyal Customers, Potential Loyalists gibi bizim istediğimiz sınıfa kayabilir. Sınırlı süreli kampanyalar yapabilir, geçmiş alışverişlerine göre tavsiyelerde bulunabilir yeniden etkinleştirebiliriz.

- **New Customers**

New Customers sınıfı incelendiğinde:

-- Receny ortalaması yaklaşık 5, sisteme yeni girmiş çok güzel,

-- Frequency ortalaması yaklaşık 7, çok az alışveriş yapmış, (bu sınıf hiç alışveriş yapmayıp sadece sisteme de girmiş yapmış olabilir.)

-- Monetary ortalaması 3742.

    -- Bu sınıf teması en son teması olan sınıftır ve genelde ya hiç alışveriş yapmazlar ya da çok az alışveriş yaparlar. Her zaman sık alışveriş yapmasını ve arayı açmaması isteriz. Frequency iyiyse bu gruba birgün pahalı üründe satabiliriz. Bunlara yeni destekler sağlayıp, iletişim kurmaya başlamamız gerekmektedir.

Yardıma ihtiyacı olan sınıfı ilgili departmanlara göndermek için "mailing" işlemi yapmak gerekir. Sadece ilgilenilen segment için idleri alıp bir dosyaya kaydetmek gerekir.

In [105]:
# Örnek olarak "Need Attention" sınıfını seçelim.
rfm[rfm["Segment"] == "Need Attention"].head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE,Segment
Customer ID,,,,,,,,
12372,70,52,1298,3,3,4,334,Need Attention
12413,65,38,758,3,3,3,333,Need Attention
12446,56,58,1002,3,3,4,334,Need Attention
12458,70,38,947,3,3,4,334,Need Attention
12475,52,36,723,3,3,3,333,Need Attention


In [107]:
new_df = pd.DataFrame()
new_df["Need AttentionID"] = rfm[rfm["Segment"] == "Need Attention"].index
new_df.head()

,Need AttentionID
0,12372
1,12413
2,12446
3,12458
4,12475


In [258]:
# Bir csv dosyasına kaydettik. Bu işlemden sonra "need_attention" adında bir csv dosyayı oluşacaktır. 
# Bu ilgili departmanlara gönderilecek dosyadır.
new_df.to_csv("need_attention.csv")

## TEŞEKKÜRLER! :)